Instalação e importação de módulos

In [ ]:
!pip install -q transformers torch accelerate

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import clear_output

Antes de prosseguir, verificar se está com a GPU habilitada. A inferência demora demais se estiver usando apenas CPU.

In [ ]:
print("GPU disponível?", torch.cuda.is_available())

if not torch.cuda.is_available():
    print("Ative a GPU em Runtime -> Change runtime type -> T4 GPU.")

GPU disponível? True


Carregando o modelo Qwen2.5-Coder-1.5B-Instruct do HuggingFace

In [ ]:
model_id = "Qwen/Qwen2.5-Coder-1.5B-Instruct"

print(f"Carregando {model_id}...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.float16,
    device_map="cuda"
)

Carregando Qwen/Qwen2.5-Coder-1.5B-Instruct...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Definições de funções úteis: **Geração de árvore de diretórios a partir do repositório clonado** e **função de inferência** com prompt.

In [ ]:
def gerar_arvore_diretorios(caminho_raiz, max_depth=3, ignorar=['.git', 'node_modules', 'dist', 'build', 'coverage', 'venv', '.github', 'assets']):
    tree_str = ""
    root_level = caminho_raiz.count(os.sep)

    for root, dirs, files in os.walk(caminho_raiz):
        dirs[:] = [d for d in dirs if d not in ignorar]
        level = root.count(os.sep) - root_level
        if level > max_depth:
            continue

        indent = ' ' * 4 * level
        tree_str += f"{indent}{os.path.basename(root)}/\n"

        if level < max_depth:
            subindent = ' ' * 4 * (level + 1)
            for f in files[:10]:
                tree_str += f"{subindent}{f}\n"
            if len(files) > 10:
                tree_str += f"{subindent}... (+{len(files)-10} arquivos)\n"

    return tree_str

def ler_readme():
  ...

def inferir_arquitetura_pela_tree(tree_text, prompt):
    messages = [
        {"role": "system", "content": "Você é um especialista em análise de código e padrões arquiteturais."},
        {"role": "user", "content": prompt}
    ]

    inputs = tokenizer.apply_chat_template(messages, tokenize=True, return_dict=True, return_tensors="pt", add_generation_prompt=True).to(model.device)

    generated_ids = model.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.2
    )

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].split("assistant")[-1].strip()

Clonando repositório objeto da análise

In [ ]:
!git clone https://github.com/Mintplex-Labs/anything-llm.git

fatal: destination path 'anything-llm' already exists and is not an empty directory.


Definição do prompt

In [ ]:
prompt = """
Aja como um Arquiteto de Software Sênior.
Analise a estrutura de arquivos (Tree) abaixo.

TAREFA:
Classifique a arquitetura deste projeto escolhendo dentre estes padrões:
[Camadas, Pipe-and-Filters, Client-Server, Peer-to-Peer, Microservices, Blockchain, SOA, Publish-Subscribe, Shared-Data, Blackboard].

IMPORTANTE:
Você DEVE fornecer 5 palpites diferentes, com uma porcentagem de confiança para cada um.

Siga ESTRITAMENTE este formato de resposta:

1. [Nome do Padrão] - [XX]%
   Justificativa: [Breve explicação baseada em pastas específicas]

2. [Nome do Padrão] - [XX]%
   Justificativa: [...]

(Repita até o 5)

--- INÍCIO DA TREE ---
{}
--- FIM DA TREE ---
"""

Por fim, executamos o modelo e esperamos pelo resultado da análise (aprox. 1min)

In [ ]:
repo_path = "./anything-llm"
tree_visual = gerar_arvore_diretorios(repo_path)

print(f"Estrutura capturada (primeiras 10 linhas):\n{'\n'.join(tree_visual.splitlines()[:10])}...\n")

print("Aguardando análise do modelo...")

resultado = inferir_arquitetura_pela_tree(tree_visual, prompt.format(tree_visual))

clear_output(wait=True)

print("Análise gerada:")
print(resultado)

Análise gerada:
Claro, vou analisar a estrutura de arquivos e classificar a arquitetura deste projeto usando os padrões mencionados. Aqui estão meus palpites com suas respectivas porcentagens de confiança:

1. **Pipe-and-Filters** - 20%
   Justificativa: O projeto usa um pipeline de processamento que envolve várias fases de transformação e filtragem de dados, conforme mostrado nas pastas `server`, `utils`, e `middleware`.

2. **Client-Server** - 30%
   Justificativa: A arquitetura é dividida em duas camadas principais: o cliente (front-end) e o servidor (back-end). O cliente envia solicitações ao servidor, que processa as informações e retorna resultados.

3. **Peer-to-Peer** - 10%
   Justificativa: Não há evidências de comunicação entre peers no projeto. Todos os componentes são independentes e operam em sua própria instância.

4. **Microservices** - 10%
   Justificativa: O projeto é dividido em vários microserviços, cada um com sua responsabilidade específica. Por exemplo, o `server`